In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/interbank20/sample_submission.csv
/kaggle/input/interbank20/y_train.csv
/kaggle/input/interbank20/sunat_test.csv
/kaggle/input/interbank20/rcc_test.csv
/kaggle/input/interbank20/se_test.csv
/kaggle/input/interbank20/sunat_train.csv
/kaggle/input/interbank20/se_train.csv
/kaggle/input/interbank20/censo_test.csv
/kaggle/input/interbank20/productos.csv
/kaggle/input/interbank20/censo_train.csv
/kaggle/input/interbank20/rcc_train.csv


In [2]:
rcc_train = pd.read_csv('/kaggle/input/interbank20/rcc_train.csv')
rcc_test = pd.read_csv('/kaggle/input/interbank20/rcc_test.csv')
y_train = pd.read_csv('/kaggle/input/interbank20/y_train.csv', index_col='key_value')

In [3]:
bins = [-1, 0, 10, 20, 30, 60, 90, 180, 360, 720, float("inf")]
rcc_train["condicion"] = pd.cut(rcc_train.condicion, bins, labels = range(0,10)).astype('int32')
rcc_test["condicion"] = pd.cut(rcc_test.condicion, bins, labels = range(0,10)).astype('int32')
rcc_test['cod_instit_financiera'].fillna(rcc_test['cod_instit_financiera'].value_counts().index[0], inplace=True)
rcc_test['PRODUCTO'].fillna(rcc_test['PRODUCTO'].value_counts().index[0], inplace=True)

In [4]:
dict_ = dict(zip(list(rcc_train.dtypes[rcc_train.dtypes == int].index), ['int32']*len(rcc_train.dtypes[rcc_train.dtypes == int].index)))
rcc_train = rcc_train.astype(dict_)
rcc_test = rcc_test.astype(dict_)

In [5]:
import statistics as stat
import numpy as np
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda

In [6]:
from tqdm.notebook import tqdm 
import gc
list_rcc_train_agg = []

moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_rcc = {'condicion':['max'],
           'saldo':['max'],
           'cod_instit_financiera':['nunique',moda],
           'PRODUCTO':['nunique',moda],
           'COD_CLASIFICACION_DEUDOR':['nunique','max','min','sum',moda]
          }

In [7]:
for n,i in enumerate(tqdm(sorted(set(rcc_train.codmes),reverse=True))):
#     if i in [201802,201712,201709,201703]:
    if n in [0,1,2,3,4,5,11]:
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldo_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
            gc.collect()
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).condicion.max().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_condicion_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
            gc.collect()

        print(f'haciendo agg ')
        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby('key_value').agg(agg_rcc)
        rcc_train_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()
        
rcc_train_ = pd.concat(list_rcc_train_agg, axis=1)
del rcc_train, list_rcc_train_agg

haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haci

In [8]:
from tqdm.notebook import tqdm 
list_rcc_test_agg = []
for n,i in enumerate(tqdm(sorted(set(rcc_test.codmes),reverse=True))):
#     if i in [201902,201812,201809,201803]:
    if n in [0,1,2,3,4,5,11]:
        for c in ['tipo_credito','cod_instit_financiera','PRODUCTO','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldo_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
            gc.collect()
        for c in ['tipo_credito','cod_instit_financiera','PRODUCTO','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).condicion.max().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_condicion_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
            gc.collect()
        print(f'haciendo agg')    
        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby('key_value').agg(agg_rcc)
        rcc_test_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

rcc_test_ = pd.concat(list_rcc_test_agg, axis=1)
del rcc_test, list_rcc_test_agg

haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo cod_instit_financiera
hacie

In [9]:
train = rcc_train_.copy()
test = rcc_test_.copy()
del rcc_train_, rcc_test_

In [10]:
### imputar datos categoricos
var_categ = [i for i in train.columns if 'mode' in i]
for i in var_categ:
    train[i] = train[i].fillna(train[i].value_counts().index[0])
    test[i] = test[i].fillna(train[i].value_counts().index[0])

In [11]:
### mean encoding
### ojo, en el test hay missing values pero pocos
train = pd.concat([train, y_train], axis=1)
for i in var_categ:
    me = train.groupby([i])['target'].mean().to_dict() 
    train[f'{i}_me'] =  train[i].map(me) 
    test[f'{i}_me'] =  test[i].map(me)
    ### imputar datos faltantes
    test[f'{i}_me'] =  test[f'{i}_me'].fillna(train[f'{i}_me'].median())
    
train.drop('target', axis=1, inplace=True)
train.drop(var_categ, axis=1, inplace=True)
test.drop(var_categ, axis=1, inplace=True)

In [84]:
se_train = pd.read_csv('/kaggle/input/interbank20/se_train.csv', index_col='key_value')
se_test = pd.read_csv('/kaggle/input/interbank20/se_test.csv', index_col='key_value')
# edad y cod_ubi tienen valores nulos con cerca de 1% cada uno

In [85]:
dict_ = {'sexo':'int32',
 'est_cvl':'int32',
 'sit_lab':'int32',
 'cod_ocu':'int32',
 'ctd_hijos':'int32',
 'flg_sin_email':'int32',
 'ctd_veh':'int32',
 'lgr_vot':'int32',
 'prv':'int32',
 'dto':'int32',
 'rgn':'int32',
 'tip_lvledu':'int32'}
se_train = se_train.astype(dict_)
se_test = se_test.astype(dict_)

In [86]:
### imputar datos faltantes en la base se_[train/test]
se_train['edad'].fillna(se_train['edad'].median(), inplace=True)
se_test['edad'].fillna(se_train['edad'].median(), inplace=True)
se_train['cod_ubi'].fillna(se_train['cod_ubi'].median(), inplace=True)
se_test['cod_ubi'].fillna(se_train['cod_ubi'].median(), inplace=True)

In [87]:
var_categ=['sit_lab','lgr_vot','sexo','rgn','tip_lvledu','flg_sin_email','prv','est_cvl','cod_ocu','dto']
# var_caunti=['edad','ctd_hijos','ctd_veh','cod_ubi']
se_train['target'] = y_train.loc[se_train.index]
for i in var_categ:
    me = se_train.groupby([i])['target'].mean().to_dict() 
    se_train[f'{i}_me'] =  se_train[i].map(me) 
    se_test[f'{i}_me'] =  se_test[i].map(me)
    ### imputar datos faltantes
    se_test[f'{i}_me'] =  se_test[f'{i}_me'].fillna(se_train[f'{i}_me'].median())
se_train.drop('target', axis=1, inplace=True)
se_train.drop(var_categ, axis=1, inplace=True)
se_test.drop(var_categ, axis=1, inplace=True)

In [90]:
train = train.join(se_train) ### Ojo 57 valores perdidos
test = test.join(se_test)
del se_train, se_test
# train.drop(list(se_train.columns), axis=1, inplace=True)
# test.drop(list(se_train.columns), axis=1, inplace=True)

In [94]:
### imputar valores faltantes
var_caunti = ['lgr_vot_me','sit_lab_me','cod_ubi','tip_lvledu_me','prv_me','rgn_me',
 'ctd_hijos','edad','cod_ocu_me','ctd_veh','est_cvl_me','dto_me','flg_sin_email_me','sexo_me']
for i in var_caunti:
    train[i] = train[i].fillna(train[i].median())
    test[i] = test[i].fillna(train[i].median())

In [97]:
### asegurar que todas la columnas aparezcan tanto en el train como en el test
print(train.shape)
keep_cols = list(set(train.columns).intersection(set(test.columns)))
train = train[keep_cols].copy()
test = test[keep_cols].copy()
print(train.shape)

(358487, 357)
(358487, 357)


In [98]:
# !pip install catboost
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import numpy as np
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = CatBoostClassifier(n_estimators=1000,  eval_metric = 'AUC', max_depth = 6)
    learner.fit(Xt, yt,  early_stopping_rounds=10, 
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50) # cat_features = var_categ_index,
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
    gc.collect()
test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))]))

********** 0 **********
Learning rate set to 0.128212
0:	test: 0.7347999	test1: 0.7356800	best: 0.7356800 (0)	total: 277ms	remaining: 4m 36s
50:	test: 0.8249635	test1: 0.8228458	best: 0.8228458 (50)	total: 14.8s	remaining: 4m 35s
100:	test: 0.8334580	test1: 0.8297161	best: 0.8297161 (100)	total: 28.1s	remaining: 4m 10s
150:	test: 0.8394486	test1: 0.8337010	best: 0.8337010 (150)	total: 41.6s	remaining: 3m 53s
200:	test: 0.8435253	test1: 0.8356967	best: 0.8356967 (200)	total: 55s	remaining: 3m 38s
250:	test: 0.8473837	test1: 0.8374114	best: 0.8374126 (249)	total: 1m 8s	remaining: 3m 23s
300:	test: 0.8500875	test1: 0.8384005	best: 0.8384005 (300)	total: 1m 21s	remaining: 3m 8s
350:	test: 0.8527908	test1: 0.8392212	best: 0.8392212 (350)	total: 1m 33s	remaining: 2m 53s
400:	test: 0.8550803	test1: 0.8398042	best: 0.8398229 (398)	total: 1m 46s	remaining: 2m 38s
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.8401143837
bestIteration = 428

Shrink model to first 429 iterati

In [ ]:
# roc auc estimado:  0.8287296558597061 benchmark

# roc auc estimado:  0.8235666881996343 solo considerando rcc, inputando datos categoricos. con 4 mes
# roc auc varianza:  0.0005453571957666847

# roc auc estimado:  0.8236751116856392 solo considerando rcc, inputando datos categoricos. con 6 mes
# roc auc varianza:  0.0007266775450365375

# roc auc estimado:  0.8240426706542976 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0006480438484228306

# roc auc estimado:  0.8358995025496325   solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.000799546878111712  y incluyendo se sin imputar datos

# roc auc estimado:  0.8362318848230975    solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0006670589830583723 y incluyendo la base se imputando edad y cod_ubi

# roc auc estimado:  0.8357779560666587  solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0008281941487650447 y incluyendo la base se imputando edad y cod_ubi. y en base final 

# roc auc estimado:  0.8386997825126665  solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007378701891037694 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding

# roc auc estimado:  0.8390378213410135 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007879483415566218 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding. Impuntando la base final

In [100]:
fi.sort_values(ascending=False).head(50)
# COD_CLASIFICACION_DEUDOR_max_ult1mes              0.054579
# COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.046220
# tipo_credito_11_saldo_ult12meses                  0.043322
# tipo_credito_11_saldo_ult1meses                   0.040848
# tipo_credito_12_saldo_ult1meses                   0.040650
# RIESGO_DIRECTO_-1_saldo_ult12meses                0.039103
# COD_CLASIFICACION_DEUDOR_0_saldo_ult1meses        0.037563
# RIESGO_DIRECTO_1_saldo_ult1meses                  0.036770

# COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.058760
# tipo_credito_11_saldo_ult12meses                  0.039706
# tipo_credito_11_saldo_ult1meses                   0.038419
# RIESGO_DIRECTO_1_saldo_ult1meses                  0.035330
# tipo_credito_12_saldo_ult1meses                   0.032713
# RIESGO_DIRECTO_-1_saldo_ult12meses                0.031817
# cod_instit_financiera_mode_ult1mes                0.029203
# cod_instit_financiera_mode_ult12mes               0.026591

# tipo_credito_11_saldo_ult12meses                  0.041308
# COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.040848
# COD_CLASIFICACION_DEUDOR_max_ult1mes              0.038907
# RIESGO_DIRECTO_1_saldo_ult1meses                  0.035023
# cod_instit_financiera_mode_ult1mes_me             0.034799
# RIESGO_DIRECTO_-1_saldo_ult12meses                0.032097
# cod_instit_financiera_mode_ult12mes_me            0.028594
# tipo_credito_12_saldo_ult1meses                   0.028358

# dto_me                                            0.060329
# COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.034688
# cod_instit_financiera_mode_ult1mes_me             0.033413
# tip_lvledu_me                                     0.031962
# tipo_credito_12_saldo_ult1meses                   0.029074
# RIESGO_DIRECTO_1_saldo_ult1meses                  0.028161
# edad                                              0.024312
# tipo_credito_11_saldo_ult12meses                  0.024109

# dto_me                                            0.064928
# tip_lvledu_me                                     0.030977
# COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.029493
# RIESGO_DIRECTO_1_saldo_ult1meses                  0.028540
# COD_CLASIFICACION_DEUDOR_sum_ult2mes              0.028324
# cod_instit_financiera_mode_ult1mes_me             0.027232
# tipo_credito_12_saldo_ult1meses                   0.027068
# COD_CLASIFICACION_DEUDOR_max_ult1mes              0.025515

dto_me                                            0.064928
tip_lvledu_me                                     0.030977
COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.029493
RIESGO_DIRECTO_1_saldo_ult1meses                  0.028540
COD_CLASIFICACION_DEUDOR_sum_ult2mes              0.028324
cod_instit_financiera_mode_ult1mes_me             0.027232
tipo_credito_12_saldo_ult1meses                   0.027068
COD_CLASIFICACION_DEUDOR_max_ult1mes              0.025515
tipo_credito_11_saldo_ult12meses                  0.024430
edad                                              0.023496
RIESGO_DIRECTO_-1_saldo_ult12meses                0.023332
tipo_credito_11_saldo_ult1meses                   0.023081
cod_instit_financiera_mode_ult12mes_me            0.019868
cod_instit_financiera_mode_ult6mes_me             0.018609
COD_CLASIFICACION_DEUDOR_0_saldo_ult1meses        0.017946
RIESGO_DIRECTO_-1_saldo_ult1meses                 0.016282
flg_sin_email_me                                  0.0160

In [ ]:
learner_ = CatBoostClassifier(n_estimators=1000, eval_metric = 'AUC', max_depth = 6)
learner_.fit(train, y_train,  early_stopping_rounds=10, verbose = 50)

Learning rate set to 0.126968
0:	total: 299ms	remaining: 4m 58s
50:	total: 15.7s	remaining: 4m 52s
100:	total: 29.5s	remaining: 4m 22s
150:	total: 44s	remaining: 4m 7s
200:	total: 57.9s	remaining: 3m 50s
250:	total: 1m 11s	remaining: 3m 34s
300:	total: 1m 24s	remaining: 3m 16s
350:	total: 1m 38s	remaining: 3m 1s
400:	total: 1m 52s	remaining: 2m 48s
450:	total: 2m 6s	remaining: 2m 34s


In [ ]:
fi_ = pd.Series(learner_.feature_importances_ / learner_.feature_importances_.sum(), index=train.columns)
fi_.sort_values(ascending=False).head(50)

In [ ]:
test_probs_ = pd.Series(learner_.predict_proba(test)[:, -1], index=test.index, name="target")
test_probs_.shape

In [ ]:
test_probs_.name = "target"
test_probs_.to_csv("test14.csv")

<a href='test14.csv'>Download csv</a>